**Note:** This is a DDL notebook. Run this only once

In [ ]:
from pyspark.sql import SparkSession

from seed.deltalake import conf

spark: SparkSession = SparkSession.builder.config(conf=conf).getOrCreate()
print(f"Spark {spark.version} is up and running!")

In [ ]:
# Create a namespace in nessie catalog

# spark.sql("DROP NAMESPACE IF EXISTS nessie.delta;")

spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.delta;")

spark.sql("SHOW NAMESPACES FROM nessie").show()

25/04/28 04:04:25 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/spark-603d6bd3-2e63-4962-915e-57859ddf4e50/userFiles-1935a4d3-4fda-44f7-8c2a-45f70ba63975. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/spark-603d6bd3-2e63-4962-915e-57859ddf4e50/userFiles-1935a4d3-4fda-44f7-8c2a-45f70ba63975
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:174)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.SparkEnv.stop(SparkEnv.scala:108)
	at org.apache.spark.SparkContext.$anonfun$stop$25(SparkContext.scala:2305)
	at org.apac

In [ ]:
# Create a test table in the namespace

spark.sql("CREATE OR REPLACE TABLE nessie.delta.names (name STRING) USING iceberg TBLPROPERTIES ('gc.enabled' = 'true');")

spark.sql("INSERT INTO nessie.delta.names VALUES ('vinay');")

spark.sql("SELECT * FROM nessie.delta.names;").show()

spark.sql("SHOW TABLES FROM nessie;").show()

# spark.sql("DROP TABLE nessie.delta.names PURGE;")

In [ ]:
# Read flights data from parquet file

df = spark.read.parquet("s3a://seed/flights-1m.parquet")
df.show(5)

df.printSchema()

df.createOrReplaceTempView("raw_flights")

spark.sql("""
    SELECT
        MIN(FL_DATE) AS min_date,
        MAX(FL_DATE) AS max_date,
        COUNT(*) AS num_rows
    FROM raw_flights;
""").show()

spark.sql("""
    SELECT
        FL_DATE,
        COUNT(*) AS num_rows
    FROM raw_flights
    GROUP BY FL_DATE
    ORDER BY FL_DATE
    LIMIT 10;
""").show()

In [ ]:
# Create flights table from parquet file

spark.sql("""
    CREATE TABLE IF NOT EXISTS nessie.delta.flights 
    USING iceberg
    PARTITIONED BY (fl_date)
    TBLPROPERTIES ('gc.enabled' = 'true')
    AS
    SELECT
        *
    FROM raw_flights;
""")

spark.sql("SELECT * FROM nessie.delta.flights LIMIT 5;").show()

# spark.sql("DROP TABLE nessie.delta.flights PURGE;")